In [ ]:
##to print bold
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

loans_2007 = pd.read_csv('../datasets/loans_2007.csv')
loans_2007.head()




In [ ]:
loans_2007.info()

After analyzing each column, we can conclude that the following features need to be removed:

id: randomly generated field by Lending Club for unique identification purposes only
member_id: also a randomly generated field by Lending Club for unique identification purposes only
funded_amnt: leaks data from the future (after the loan is already started to be funded)
funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
grade: contains redundant information as the interest rate column (int_rate)
sub_grade: also contains redundant information as the interest rate column (int_rate)
emp_title: requires other data and a lot of processing to potentially be useful
issue_d: leaks data from the future (after the loan is already completed funded)
zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
out_prncp: leaks data from the future, (after the loan already started to be paid off)
out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)
total_rec_int: leaks data from the future, (after the loan already started to be paid off),
total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
recoveries: also leaks data from the future, (after the loan already started to be paid off),
collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).



In [ ]:
loans_2007=loans_2007.drop(columns={"id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d","zip_code", 
                                    "out_prncp","out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
                                    "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"
                                   })

In [ ]:
loans_2007.head()

In [ ]:
loans_2007["loan_status"].value_counts()

The potential target columns are int_rate or loan_status

Lets start with loan status

In [ ]:
loan_stats = loans_2007["loan_status"].value_counts()

In [ ]:
loan_stats.index

In [ ]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

In [ ]:
#drop if only 1 unique val
dropcols = []
for col_name in loans_2007.columns:
    non_null = loans_2007[col_name].dropna()
    unique_non_null = non_null.unique()
    num_true_unique = len(unique_non_null)
    if num_true_unique < 2:
        dropcols.append(col_name)
dropcols

In [ ]:
for item in dropcols:
    print(item + ":")
    print(loans_2007[item].unique())

In [ ]:
loans_2007 = loans_2007.drop(columns=dropcols)

In [ ]:
loans_2007.info()

In [ ]:
loans_2007.isnull().sum()

In [ ]:
loans_2007.dropna().corr().sort_values(by="loan_status",ascending=False)

In [ ]:
loans = loans_2007
loans = loans.drop(columns=["pub_rec_bankruptcies"])

In [ ]:
loans = loans.dropna(axis=0)

In [ ]:
loans_2007.info()

# object_columns_df = loans.select_dtypes(include=['object'])
object_columns_df.head()

In [ ]:
object_columns_df = loans.select_dtypes(include=['object'])
object_columns_df.head()

In [ ]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    printmd(c + ":")
    print(loans[c].value_counts())

In [ ]:
printmd("purpose:")
print(loans["purpose"].value_counts())

printmd("title:")
print(loans["title"].value_counts())

In [ ]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}
loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)

loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

In [ ]:
dummy_cols = ['home_ownership', 'verification_status', 'purpose', 'term']
dummy_df = pd.get_dummies(loans[dummy_cols])

loans_w_d = pd.concat([loans,dummy_df], axis=1).drop(columns=dummy_cols)
loans_w_d.info()